# Importing required libraries.

In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn import tree
# sales_data = pd.read_csv("df_final.csv")

Files
* train.csv - historical data including Sales
* test.csv - historical data excluding Sales
* store.csv - supplemental information about the stores

We will combine 2 files train.csv and and store.csv under unique Id for each store

In [27]:
# Đọc dữ liệu từ file train.csv
train_data = pd.read_csv('../data/orginal_data/train.csv', low_memory=False)

# Đọc dữ liệu từ file store.csv
store_data = pd.read_csv('../data/orginal_data/store.csv', low_memory=False)

# Gộp hai DataFrame theo cột 'store'
merged_data = pd.merge(train_data, store_data, on='Store')

# # Lưu dữ liệu đã gộp vào một tệp tin mới
# merged_data.to_csv('../data/orginal_data/merged_data.csv', index=False)

In [28]:
train_data

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [29]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  object 
 11  CompetitionDistance        1014567 non-null  float64
 12  CompetitionOpenSinceMonth  693861 non-null   float64
 13  CompetitionO

Split add a new column named DayOfWeekName from the column DayOfWeek in DataFrame merged_data

In [30]:
# Tạo một từ điển ánh xạ giữa giá trị DayOfWeek và tên ngày tương ứng
day_of_week_map = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}

# Tạo cột mới DayOfWeekName bằng cách ánh xạ giá trị từ cột DayOfWeek
merged_data['DayOfWeekName'] = merged_data['DayOfWeek'].map(day_of_week_map)


Split one more column from merged_data column MonthName from column Date

In [31]:
# Tạo một cột mới MonthName từ cột Date
merged_data['MonthName'] = pd.to_datetime(merged_data['Date']).dt.strftime('%b')

In [32]:
merged_data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayOfWeekName,MonthName
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.00,9.00,2008.00,0,NaN,NaN,NaN,Friday,Jul
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.00,9.00,2008.00,0,NaN,NaN,NaN,Thursday,Jul
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.00,9.00,2008.00,0,NaN,NaN,NaN,Wednesday,Jul
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.00,9.00,2008.00,0,NaN,NaN,NaN,Tuesday,Jul
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.00,9.00,2008.00,0,NaN,NaN,NaN,Monday,Jul


In [33]:
# Thiết lập tùy chọn hiển thị để giá trị là số thực
pd.set_option('display.float_format', '{:.2f}'.format)
merged_data.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,1017209.00,1017209.00,1017209.00,1017209.00,1017209.00,1017209.00,1017209.00,1014567.00,693861.00,693861.00,1017209.00,509178.00,509178.00
mean,558.43,4.00,5773.82,633.15,0.83,0.38,0.18,5430.09,7.22,2008.69,0.50,23.27,2011.75
std,321.91,2.00,3849.93,464.41,0.38,0.49,0.38,7715.32,3.21,5.99,0.50,14.10,1.66
min,1.00,1.00,0.00,0.00,0.00,0.00,0.00,20.00,1.00,1900.00,0.00,1.00,2009.00
25%,280.00,2.00,3727.00,405.00,1.00,0.00,0.00,710.00,4.00,2006.00,0.00,13.00,2011.00
50%,558.00,4.00,5744.00,609.00,1.00,0.00,0.00,2330.00,8.00,2010.00,1.00,22.00,2012.00
75%,838.00,6.00,7856.00,837.00,1.00,1.00,0.00,6890.00,10.00,2013.00,1.00,37.00,2013.00
max,1115.00,7.00,41551.00,7388.00,1.00,1.00,1.00,75860.00,12.00,2015.00,1.00,50.00,2015.00


In [34]:
# Sắp xếp DataFrame theo cột Date
sorted_merged_data = merged_data.sort_values('Date')
# Sắp xếp DataFrame theo cột id theo thứ tự giảm dần
sorted_merged_data = sorted_merged_data.sort_values('Store', ascending=False)
sorted_merged_data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayOfWeekName,MonthName
1017208,1115,2,2013-01-01,0,0,0,0,a,1,d,c,5350.00,NaN,NaN,1,22.00,2012.00,"Mar,Jun,Sept,Dec",Tuesday,Jan
1016801,1115,3,2014-02-12,3982,319,1,0,0,0,d,c,5350.00,NaN,NaN,1,22.00,2012.00,"Mar,Jun,Sept,Dec",Wednesday,Feb
1016979,1115,7,2013-08-18,0,0,0,0,0,0,d,c,5350.00,NaN,NaN,1,22.00,2012.00,"Mar,Jun,Sept,Dec",Sunday,Aug
1016479,1115,3,2014-12-31,3847,290,1,0,0,1,d,c,5350.00,NaN,NaN,1,22.00,2012.00,"Mar,Jun,Sept,Dec",Wednesday,Dec
1016745,1115,3,2014-04-09,4531,322,1,0,0,0,d,c,5350.00,NaN,NaN,1,22.00,2012.00,"Mar,Jun,Sept,Dec",Wednesday,Apr


In [35]:
sorted_merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1017209 entries, 1017208 to 0
Data columns (total 20 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  object 
 11  CompetitionDistance        1014567 non-null  float64
 12  CompetitionOpenSinceMonth  693861 non-null   float64
 13  CompetitionOpenSi

# Cleaning dataset.

In [36]:
sales_data = sorted_merged_data

In [37]:
sales_data.isnull().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance            2642
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
DayOfWeekName                     0
MonthName                         0
dtype: int64

In [38]:
cat_col = []
for x in sales_data.dtypes.index:
  if sales_data.dtypes[x] == 'object':
    cat_col.append(x)
cat_col

['Date',
 'StateHoliday',
 'StoreType',
 'Assortment',
 'PromoInterval',
 'DayOfWeekName',
 'MonthName']

In [39]:
cat_col.remove('DayOfWeekName')
cat_col.remove('Date')
cat_col

['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval', 'MonthName']

In [40]:
for col in cat_col:
  print(col)
  print(sales_data[col].value_counts())
  print()

StateHoliday
StateHoliday
0    986159
a     20260
b      6690
c      4100
Name: count, dtype: int64

StoreType
StoreType
a    551627
d    312912
c    136840
b     15830
Name: count, dtype: int64

Assortment
Assortment
a    537445
c    471470
b      8294
Name: count, dtype: int64

PromoInterval
PromoInterval
Jan,Apr,Jul,Oct     293122
Feb,May,Aug,Nov     118596
Mar,Jun,Sept,Dec     97460
Name: count, dtype: int64

MonthName
MonthName
May    103695
Mar    103695
Jan    103694
Apr    100350
Jun    100350
Jul     98115
Feb     93660
Aug     63550
Dec     63550
Oct     63550
Nov     61500
Sep     61500
Name: count, dtype: int64



# Filling null values


In [41]:
sales_data['CompetitionOpenSinceMonth'] = sales_data['CompetitionOpenSinceMonth'].fillna(method='ffill').fillna(method='bfill')
sales_data['CompetitionOpenSinceYear'] = sales_data['CompetitionOpenSinceYear'].fillna(method='ffill').fillna(method='bfill')
sales_data['CompetitionDistance'] = sales_data['CompetitionDistance'].fillna(method='ffill').fillna(method='bfill')
sales_data['Promo2SinceWeek'] = sales_data['Promo2SinceWeek'].fillna(method='ffill').fillna(method='bfill')
sales_data['Promo2SinceYear'] = sales_data['Promo2SinceYear'].fillna(method='ffill').fillna(method='bfill')
sales_data['PromoInterval'] = sales_data['PromoInterval'].fillna(method='ffill').fillna(method='bfill')
sales_data.isnull().sum()
sales_data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayOfWeekName,MonthName
1017208,1115,2,2013-01-01,0,0,0,0,a,1,d,c,5350.00,4.00,2006.00,1,22.00,2012.00,"Mar,Jun,Sept,Dec",Tuesday,Jan
1016801,1115,3,2014-02-12,3982,319,1,0,0,0,d,c,5350.00,4.00,2006.00,1,22.00,2012.00,"Mar,Jun,Sept,Dec",Wednesday,Feb
1016979,1115,7,2013-08-18,0,0,0,0,0,0,d,c,5350.00,4.00,2006.00,1,22.00,2012.00,"Mar,Jun,Sept,Dec",Sunday,Aug
1016479,1115,3,2014-12-31,3847,290,1,0,0,1,d,c,5350.00,4.00,2006.00,1,22.00,2012.00,"Mar,Jun,Sept,Dec",Wednesday,Dec
1016745,1115,3,2014-04-09,4531,322,1,0,0,0,d,c,5350.00,4.00,2006.00,1,22.00,2012.00,"Mar,Jun,Sept,Dec",Wednesday,Apr


In [42]:
sales_data.isnull().sum()

Store                        0
DayOfWeek                    0
Date                         0
Sales                        0
Customers                    0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
DayOfWeekName                0
MonthName                    0
dtype: int64

In [43]:
# Lưu dữ liệu đã gộp vào một tệp tin mới
sales_data.to_csv('../data/df_final.csv', index=False)

In [44]:
sales_data

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayOfWeekName,MonthName
1017208,1115,2,2013-01-01,0,0,0,0,a,1,d,c,5350.00,4.00,2006.00,1,22.00,2012.00,"Mar,Jun,Sept,Dec",Tuesday,Jan
1016801,1115,3,2014-02-12,3982,319,1,0,0,0,d,c,5350.00,4.00,2006.00,1,22.00,2012.00,"Mar,Jun,Sept,Dec",Wednesday,Feb
1016979,1115,7,2013-08-18,0,0,0,0,0,0,d,c,5350.00,4.00,2006.00,1,22.00,2012.00,"Mar,Jun,Sept,Dec",Sunday,Aug
1016479,1115,3,2014-12-31,3847,290,1,0,0,1,d,c,5350.00,4.00,2006.00,1,22.00,2012.00,"Mar,Jun,Sept,Dec",Wednesday,Dec
1016745,1115,3,2014-04-09,4531,322,1,0,0,0,d,c,5350.00,4.00,2006.00,1,22.00,2012.00,"Mar,Jun,Sept,Dec",Wednesday,Apr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,1,1,2013-01-28,4055,549,1,0,0,0,c,a,1270.00,9.00,2008.00,0,13.00,2010.00,"Jan,Apr,Jul,Oct",Monday,Jan
341,1,7,2014-08-24,0,0,0,0,0,1,c,a,1270.00,9.00,2008.00,0,13.00,2010.00,"Jan,Apr,Jul,Oct",Sunday,Aug
221,1,1,2014-12-22,9331,1080,1,0,0,1,c,a,1270.00,9.00,2008.00,0,13.00,2010.00,"Jan,Apr,Jul,Oct",Monday,Dec
222,1,7,2014-12-21,0,0,0,0,0,0,c,a,1270.00,9.00,2008.00,0,13.00,2010.00,"Jan,Apr,Jul,Oct",Sunday,Dec


# Transforming categorical data.

In [45]:
le = LabelEncoder()
sales_data['MonthName'] = le.fit_transform(sales_data['MonthName'].astype(str))
for col in cat_col:
    sales_data[col] = le.fit_transform(sales_data[col].astype(str))
sales_data['DayOfWeekName'] = le.fit_transform(sales_data['DayOfWeekName'].astype(str))

In [46]:
sales_data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayOfWeekName,MonthName
1017208,1115,2,2013-01-01,0,0,0,0,1,1,3,2,5350.00,4.00,2006.00,1,22.00,2012.00,2,5,6
1016801,1115,3,2014-02-12,3982,319,1,0,0,0,3,2,5350.00,4.00,2006.00,1,22.00,2012.00,2,6,5
1016979,1115,7,2013-08-18,0,0,0,0,0,0,3,2,5350.00,4.00,2006.00,1,22.00,2012.00,2,3,1
1016479,1115,3,2014-12-31,3847,290,1,0,0,1,3,2,5350.00,4.00,2006.00,1,22.00,2012.00,2,6,4
1016745,1115,3,2014-04-09,4531,322,1,0,0,0,3,2,5350.00,4.00,2006.00,1,22.00,2012.00,2,6,0


# Splitting into train and test dataset form training.

In [47]:
X = sales_data.drop(columns=['Store','Date','Sales', 'Promo2', 'Open', 'CompetitionOpenSinceYear','DayOfWeekName' ,'Promo2SinceYear'])
y = sales_data['Sales']

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.1, random_state=1)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
c=0
for i in Y_test:
  if i==0:
    c=c+1
c


(915488, 12) (101721, 12) (915488,) (101721,)


17187

In [49]:
model = tree.DecisionTreeRegressor(max_depth= 8)
model.fit(X_train, Y_train)
model.score(X_train, Y_train)

0.9175910291941574

In [50]:
predictions = model.predict([[6,2,437,0,0,0,0,0,4520.0,2.0,14.0, 1]])
predictions = np.around(predictions)
predictions

c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([0.])

# Saving the model.

In [51]:
import pickle
filename = 'sales_prediction.sav'
pickle.dump(model, open(filename, 'wb'))